# 🛠️ Function/Tool Calling with OpenAI  

In this notebook you’ll learn how to connect LLMs to **external tools and APIs**.  
Instead of only generating text, the model can:  
- Decide when to call a tool (e.g., weather, FX, web search).  
- Pass structured arguments to your function.  
- Receive back results (JSON, text, numbers, etc.).  
- Combine tool outputs into a final, natural-language answer.  

We’ll explore step-by-step:  
1. **Basic tool calls** — define a schema and inspect results.  
2. **Dummy backends** — simulate weather/FX lookups.  
3. **Parallel tool calls** — model chooses multiple tools at once.  
4. **Real APIs** — Open-Meteo for weather, ExchangeRate API for FX.  
5. **Web search** — using DuckDuckGo / Tavily for live information.  
6. **Multiple tools together** — model selects which tool(s) to use.  



In [ ]:
!pip install requests

In [ ]:

from openai import OpenAI
import json, requests, ast, os, operator as op

os.environ["OPENAI_API_KEY"] = "" 

client = OpenAI()

MODEL = "gpt-4o"



/Users/tanmaydhote/Documents/src/ai-accelerator-walmart-scratch/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## 1) Defining and Using a Simple Tool  

This example shows the **basics of function calling** with an LLM.  

1. **Define a tool schema** — tell the model what function exists (`get_weather`), what it does, and the arguments it requires (JSON Schema).  
2. **Call the model with the tool available** — we pass a natural language question (“What is weather in Paris?”). The model can either answer directly in text or decide to call the tool.  
3. **Inspect the response** — if the model chooses to call the tool, the call appears in `response.choices[0].message.tool_calls`.  


In [12]:
# ---------- 1) Define the tool schema ----------
# Define a tool that the model can "call" when it needs weather info.
# This tells the model:
# - tool name ("get_weather")
# - description (helps the model know when to use it)
# - parameters (what arguments the tool expects, in JSON Schema form)
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get current weather for a city",
            "parameters": {
                "type": "object",       
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "City for which to get the current weather, e.g. Paris"
                    }
                },
                "required": [
                    "city"                # city argument is mandatory
                ]
            }
        }
    }
]


# ---------- 2)  Model call ----------
# We ask the model a natural-language question.
# Because we provide `tools=tools`, the model has the option to respond
# either with a normal text answer OR by calling the tool.
response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {
            "role": "user",
            "content": "What is weather in Paris?"
        }
    ],
    tools=tools
)

# ---------- 3) Inspect the result ----------
# The model may decide to call the tool, in which case
# `response.choices[0].message.tool_calls` will contain that request.
# For now we just print the raw message to see what the model chose.
print(response.choices[0].message)

# ---------- 4) Model call with a query that doesn't require a tool----------

response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {
            "role": "user",
            "content": "What is language spoken in Paris?"
        }
    ],
    tools=tools
)

# ---------- 5) Inspect the result ----------
# We just print the raw message to see what the model chose.
print(response.choices[0].message)


ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_jN2AJahy5JMeLVpFyngRJOTm', function=Function(arguments='{"city":"Paris"}', name='get_weather'), type='function')])
ChatCompletionMessage(content='The primary language spoken in Paris is French.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)


## 2) Handling a Tool Call with a Dummy Function  

In the previous step, the model could *decide* to call a tool (`get_weather`).  
Now we’ll **simulate a backend** by:  
1. Defining a simple Python function `get_weather` that returns static results.  
2. Detecting when the model calls the tool.  
3. Running our dummy function to produce the output.  
4. Sending the tool’s result back to the model so it can give a final answer.  

In [42]:
import json

# ---------- 1) Define a dummy weather lookup ----------
def get_weather(city: str) -> str:
    # Pretend this is a weather database
    weather_data = {
        "Paris": "15°C, cloudy",
        "London": "10°C, rainy",
        "New York": "22°C, sunny"
    }
    return weather_data.get(city, "Weather data not available")

# ---------- 2) Ask the model (same as 1A) ----------
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get current weather for a city",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "City for which to get the current weather, e.g. Paris"
                    }
                },
                "required": ["city"]
            }
        }
    }
]

response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "user", "content": "What is weather in Paris?"}
    ],
    tools=tools
)

# ---------- 3) Inspect if tool was called ----------
message = response.choices[0].message

if message.tool_calls:
    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_weather":
            args = json.loads(tool_call.function.arguments)
            city = args["city"]

            # Run the dummy function
            result = get_weather(city)

            # ---------- 4) Send the result back to the model ----------
            followup = client.chat.completions.create(
                model=MODEL,
                messages=[
                    {"role": "user", "content": "What is weather in Paris?"},
                    message,  # include the original tool call
                    {
                        "role": "tool",
                        "tool_call_id": tool_call.id,
                        "content": result
                    }
                ]
            )

            print(followup.choices[0].message.content)


The current weather in Paris is 15°C with cloudy skies.


## 3) Parallel Tool Calls (Comparing Cities)  

The model isn’t limited to calling a tool once — it can request **multiple tool calls in parallel**.  
In this example, we ask: *“Compare Paris vs London weather and recommend one for a picnic.”*  

- The model issues two tool calls (`get_weather` for Paris and London).  
- We execute each dummy lookup and return their results.  
- A second model call combines the outputs into a natural-language comparison and recommendation.  


In [18]:
import json

# ---------- 1) Define a dummy weather lookup ----------
def get_weather(city: str) -> str:
    # Pretend this is a weather database
    weather_data = {
        "Paris": "15°C, cloudy",
        "London": "10°C, rainy",
        "New York": "22°C, sunny"
    }
    return weather_data.get(city, "Weather data not available")

# --- Tool schema (function calling) ---
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get current weather for a city",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {"type": "string", "description": "City name, e.g., Paris"}
                },
                "required": ["city"]
            }
        }
    }
]

# --- Initial prompt: compare two cities and recommend one for a picnic ---
user_prompt = "Compare Paris vs London weather and recommend one for a picnic."

# First call: let the model decide to call the tool (likely twice: Paris & London)
messages = [{"role": "user", "content": user_prompt}]
first = client.chat.completions.create(model=MODEL, messages=messages, tools=tools)

msg = first.choices[0].message
messages.append(msg)  # keep transcript

#print("Output of first call: ", msg)

# If the model made tool calls, run them and add results to the transcript
if msg.tool_calls:
    for call in msg.tool_calls:
        if call.function.name == "get_weather":
            args = json.loads(call.function.arguments or "{}")
            city = args.get("city", "")
            result = get_weather(city)
            messages.append({
                "role": "tool",
                "tool_call_id": call.id,
                "name": "get_weather",
                "content": result
            })

# Second call: model reads tool outputs and gives a natural-language comparison + recommendation
#print("Input to second call: ", messages)
final = client.chat.completions.create(model=MODEL, messages=messages)
print(final.choices[0].message.content)

Based on the current weather conditions:

- **Paris**: 15°C, cloudy
- **London**: 10°C, rainy

Given these conditions, Paris would be the better choice for a picnic. The temperature is warmer, and although it is cloudy, it is not raining. In contrast, London is cooler and experiencing rain, which would not be ideal for outdoor activities like a picnic. Enjoy your time in Paris!


## 4) Using a Real API — Weather (Open-Meteo)  

So far we’ve used dummy lookups. Now let’s connect a tool to a **real API**.  
Here, the model calls `get_weather(city)`, which:  
1. Uses Open-Meteo’s geocoding API to find latitude/longitude.  
2. Calls Open-Meteo’s forecast API to get the current weather.  
3. Returns structured JSON (temperature, windspeed, etc.).  

The model first decides whether to call the tool.  
If it does, we run the API function and send the result back for a final natural-language answer.  


In [14]:
import requests, json

# ---------- 1) Define a real weather lookup ----------
def get_weather(city: str):
    # Step 1: Convert city name → coordinates
    geo_resp = requests.get(
        "https://geocoding-api.open-meteo.com/v1/search",
        params={"name": city, "count": 1}
    ).json()
    if not geo_resp.get("results"):
        return {"ok": False, "error": "City not found"}
    
    lat = geo_resp["results"][0]["latitude"]
    lon = geo_resp["results"][0]["longitude"]

    # Step 2: Fetch current weather for that location
    weather_resp = requests.get(
        "https://api.open-meteo.com/v1/forecast",
        params={"latitude": lat, "longitude": lon, "current_weather": True}
    ).json()

    return {
        "ok": True,
        "city": city,
        "current_weather": weather_resp.get("current_weather", {})
    }

# ---------- 2) Define tool schema ----------
tools_weather = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get current weather for a city",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {"type": "string"}
                },
                "required": ["city"]
            }
        }
    }
]

# ---------- 3) Ask the model ----------
user_query = "What’s the weather in Dallas right now?"

initial_response = client.chat.completions.create(
    model=MODEL,
    messages=[{"role": "user", "content": user_query}],
    tools=tools_weather,

    # tool_choice options:
    # - "auto" (default): model decides whether to call a tool or answer directly
    # - "none": model is not allowed to call any tools (must answer in text)
    # - force a specific tool: pass a dict like below
    #   {"type": "function", "function": {"name": "get_weather"}}
    tool_choice="auto"
)

assistant_message = initial_response.choices[0].message

# ---------- 4) Handle tool call if present ----------
if assistant_message.tool_calls:
    tool_call = assistant_message.tool_calls[0]
    call_args = json.loads(tool_call.function.arguments)

    # Run the real API function
    tool_output = get_weather(call_args["city"])

    # Create a tool message with the API result
    tool_message = {
        "tool_call_id": tool_call.id,
        "role": "tool",
        "name": "get_weather",
        "content": json.dumps(tool_output)
    }

    # ---------- 5) Send API result back to model ----------
    final_response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "user", "content": user_query},
            assistant_message,
            tool_message
        ]
    )

    print(final_response.choices[0].message.content)


The current weather in Dallas is 31.8°C with a wind speed of 8.5 km/h coming from the southeast (152°). It is daytime and clear weather conditions are noted.


## 5) Multiple Tools — Model Chooses (Dummy Weather + Dummy FX)

Expose **two tools** at once:  
- a dummy `get_weather(city)` lookup  
- a dummy `get_exchange_rate(base, target)`  

Ask a question that needs both (e.g., *“I am planning to visit Paris. How is the weather there currently? And what’s the exchange rate from USD to EUR?”*).  
The model decides which tool(s) to call.  

In [21]:

# ---------- 1) Dummy weather lookup ----------
def get_weather(city: str):
    weather_data = {
        "Paris": "18°C, partly cloudy",
        "London": "16°C, rainy",
        "New York": "22°C, sunny"
    }
    return {"ok": True, "city": city, "weather": weather_data.get(city, "Weather data not available")}

# ---------- 2) Dummy FX lookup ----------
def get_exchange_rate(base: str, target: str):
    fx_data = {
        ("USD", "EUR"): 0.92,
        ("USD", "INR"): 83.10,
        ("EUR", "INR"): 90.15
    }
    rate = fx_data.get((base, target))
    return {"ok": True, "base": base, "target": target, "rate": rate or "Rate not available"}

# ---------- 3) Define tool schemas ----------
tools_multi = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get current weather for a city",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {"type": "string", "description": "City, e.g., Paris"}
                },
                "required": ["city"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_exchange_rate",
            "description": "Get the current exchange rate between two currencies",
            "parameters": {
                "type": "object",
                "properties": {
                    "base": {"type": "string", "description": "Base currency, e.g., USD"},
                    "target": {"type": "string", "description": "Target currency, e.g., EUR"}
                },
                "required": ["base", "target"]
            }
        }
    }
]

# ---------- 4) Ask a question needing both ----------
user_query = (
    "I am planning to visit Paris. How is the weather there currently? "
    "And what’s the exchange rate from USD to EUR? I have 100 USD."
)

initial_response = client.chat.completions.create(
    model=MODEL,
    messages=[{"role": "user", "content": user_query}],
    tools=tools_multi,
    tool_choice="auto"   # model chooses which tool(s) to call
)

assistant_message = initial_response.choices[0].message
print("Output of first call: ", assistant_message)

# ---------- 5) Execute any/all tool calls ----------
tool_messages = []
if assistant_message.tool_calls:
    for tool_call in assistant_message.tool_calls:
        fn = tool_call.function.name
        args = json.loads(tool_call.function.arguments or "{}")

        if fn == "get_weather":
            result = get_weather(args.get("city", ""))
        elif fn == "get_exchange_rate":
            result = get_exchange_rate(args.get("base", ""), args.get("target", ""))
        else:
            result = {"ok": False, "error": f"Unknown tool: {fn}"}

        tool_messages.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "name": fn,
            "content": json.dumps(result)
        })

print("Tool messages: ", tool_messages)

# ---------- 6) Send all tool results back ----------
final_response = client.chat.completions.create(
    model=MODEL,
    messages=[{"role": "user", "content": user_query}, assistant_message, *tool_messages]
)

print(final_response.choices[0].message.content)


Output of first call:  ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_A9ZRZTfGTNhoDOnpmJ41T2va', function=Function(arguments='{"city": "Paris"}', name='get_weather'), type='function'), ChatCompletionMessageToolCall(id='call_td9nzTgpI5IZ8mhVBwHvnHlF', function=Function(arguments='{"base": "USD", "target": "EUR"}', name='get_exchange_rate'), type='function')])
Tool messages:  [{'role': 'tool', 'tool_call_id': 'call_A9ZRZTfGTNhoDOnpmJ41T2va', 'name': 'get_weather', 'content': '{"ok": true, "city": "Paris", "weather": "18\\u00b0C, partly cloudy"}'}, {'role': 'tool', 'tool_call_id': 'call_td9nzTgpI5IZ8mhVBwHvnHlF', 'name': 'get_exchange_rate', 'content': '{"ok": true, "base": "USD", "target": "EUR", "rate": 0.92}'}]
Currently, the weather in Paris is 18°C and partly cloudy. 

As for the exchange rate, 1 USD is approximately 0.92 EUR. If you have 100 USD, you would get abou

## 6) Multiple Tools — Model Chooses (Weather + FX)

Expose **two tools** at once: a real-weather lookup (Open-Meteo) and a real FX converter (exchangerate.host).  
Ask a single question that needs **both**. The model will issue tool calls for each, you run them, then send results back for a final response.


In [23]:
import json, requests

# ---------- 1) Real weather lookup (Open-Meteo) ----------
def get_weather(city: str):
    # Step 1: name -> lat/lon
    geo_resp = requests.get(
        "https://geocoding-api.open-meteo.com/v1/search",
        params={"name": city, "count": 1}
    ).json()
    if not geo_resp.get("results"):
        return {"ok": False, "error": "City not found"}
    lat = geo_resp["results"][0]["latitude"]
    lon = geo_resp["results"][0]["longitude"]

    # Step 2: current weather
    weather_resp = requests.get(
        "https://api.open-meteo.com/v1/forecast",
        params={"latitude": lat, "longitude": lon, "current_weather": True}
    ).json()

    return {
        "ok": True,
        "city": city,
        "current_weather": weather_resp.get("current_weather", {})
    }

# ---------- 2) FX rate lookup (ExchangeRate-API) ----------
def get_exchange_rate(base: str, target: str):
    fx_resp = requests.get(f"https://open.er-api.com/v6/latest/{base}").json()

    if fx_resp.get("result") != "success":
        return {"ok": False, "error": fx_resp.get("error-type", "API call failed")}
    
    rate = fx_resp.get("rates", {}).get(target)
    return {
        "ok": bool(rate),
        "base": base,
        "target": target,
        "rate": rate
    }

# ---------- 3) Declare BOTH tools to the model ----------
tools_multi = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get current weather for a city",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {"type": "string", "description": "City, e.g., Paris"}
                },
                "required": ["city"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_exchange_rate",
            "description": "Get the current exchange rate between two currencies",
            "parameters": {
                "type": "object",
                "properties": {
                    "base": {"type": "string", "description": "Base currency code, e.g., USD"},
                    "target": {"type": "string", "description": "Target currency code, e.g., EUR"}
                },
                "required": ["base", "target"]
            }
        }
    }
]

# ---------- 4) Ask a single question that needs both tools ----------
user_query = (
    "I am planning to visit Paris. How is the weather there currently? "
    "And what’s the exchange rate from USD to EUR? I have 100 USD."
)

initial_response = client.chat.completions.create(
    model=MODEL,
    messages=[{"role": "user", "content": user_query}],
    tools=tools_multi,
    tool_choice="auto"
)

assistant_message = initial_response.choices[0].message

# ---------- 5) Execute any/all tool calls ----------
tool_messages = []
if assistant_message.tool_calls:
    for tool_call in assistant_message.tool_calls:
        fn = tool_call.function.name
        args = json.loads(tool_call.function.arguments or "{}")

        if fn == "get_weather":
            result = get_weather(args.get("city", ""))
        elif fn == "get_exchange_rate":
            result = get_exchange_rate(args.get("base", ""), args.get("target", ""))
        else:
            result = {"ok": False, "error": f"Unknown tool: {fn}"}

        tool_messages.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "name": fn,
            "content": json.dumps(result)
        })

# ---------- 6) Send all tool results back ----------
final_response = client.chat.completions.create(
    model=MODEL,
    messages=[{"role": "user", "content": user_query}, assistant_message, *tool_messages]
)

print(final_response.choices[0].message.content)


Currently, the weather in Paris is quite pleasant with a temperature of 24.4°C. The windspeed is around 4.8 km/h coming from the southwest.

As for the exchange rate, 1 USD is approximately equal to 0.8588 EUR. If you have 100 USD, you would get around 85.88 EUR.


## 7) Web Search Tool — Tavily API  

You can also wire up a **web search tool**.  
Here we connect to the Tavily Search API, which is designed for LLMs and returns a concise answer with supporting sources.  

Flow:  
1. Define `web_search(query)` → calls Tavily and returns an answer + results.  
2. Provide the tool schema to the model.  
3. Ask a question that requires external knowledge.  
4. If the model calls the tool, run the API call and feed results back.  
5. Make a second model call to generate a final, natural-language answer that incorporates the search results.  


In [40]:
import requests, json, os

# ---------- 1) Define Tavily web search ----------
#NOTE:
# Most reliable web search providers (SerpAPI, DuckDuckGo Search API, Perplexity, Tavily, etc.)
# require an API key — they are paid/freemium services.
# For now, you can use the instructor’s key, but for production use
# you will need to bring your own key from the provider’s dashboard.

def web_search(query: str):
    """Call Tavily Search API and return summary + sources."""
    resp = requests.post(
        "https://api.tavily.com/search",
        headers={"Authorization": "Bearer "API-KEY"},
        json={"query": query, "max_results": 5}   # keep it concise
    ).json()

    return {
        "answer": resp.get("answer"),
        "results": [
            {
                "title": r.get("title"),
                "url": r.get("url"),
                "content": r.get("content"),
                "score": r.get("score")
            }
            for r in resp.get("results", [])
        ]
    }

# ---------- 2) Define tool schema ----------
tools_search = [
    {
        "type": "function",
        "function": {
            "name": "web_search",
            "description": "Perform a web search and return summary + sources",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string"}
                },
                "required": ["query"]
            }
        }
    }
]

# ---------- 3) Ask a question ----------
user_query = "What is the latest bitcoin price?”"

initial_response = client.chat.completions.create(
    model=MODEL,
    messages=[{"role": "user", "content": user_query}],
    tools=tools_search,
    tool_choice="auto"
)

assistant_message = initial_response.choices[0].message

# ---------- 4) Run the tool if model calls it ----------
if assistant_message.tool_calls:
    tool_call = assistant_message.tool_calls[0]
    args = json.loads(tool_call.function.arguments)

    tool_output = web_search(args["query"])
    print("Tool output: ", tool_output)
    tool_message = {
        "tool_call_id": tool_call.id,
        "role": "tool",
        "name": "web_search",
        "content": json.dumps(tool_output)
    }

    # ---------- 5) Send tool result back ----------
    final_response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "user", "content": user_query},
            assistant_message,
            tool_message
        ]
    )

    print(final_response.choices[0].message.content)


Tool output:  {'answer': None, 'results': [{'title': 'Bitcoin BTC (BTC-USD) Live Price, News, Chart & Price History', 'url': 'https://finance.yahoo.com/quote/BTC-USD/', 'content': 'Bitcoin has a current supply of 19,912,209. The last known price of Bitcoin is 110,854.49673546 USD and is up 0.70 over the last 24 hours.Historical Data·Community·Chart·ProShares UltraShort Bitcoin', 'score': 0.8405867}, {'title': 'Bitcoin price today, BTC to USD live price, marketcap and chart', 'url': 'https://coinmarketcap.com/currencies/bitcoin/', 'content': 'Rating4.4(3)The live Bitcoin price today is $111715.02 USD with a 24-hour trading volume of $61675436010.63 USD. We update our BTC to USD price in real-time.', 'score': 0.8342047}, {'title': 'Bitcoin (BTC) Price | BTC to USD Price and Live Chart - CoinDesk', 'url': 'https://www.coindesk.com/price/bitcoin', 'content': 'The price of Bitcoin (BTC) is $111981.50 today as of Aug 27, 2025, 3:25 pm EDT, with a 24-hour trading volume of $19.82B.Bitcoin Cas

## 🟣 Bonus — OpenAI Built-in Tools (Responses API)

> ⚠️ **Do Not Use Yet**  
> We’re waiting for access to the new **Responses API** endpoint on Azure OpenAI.  
> This is the only endpoint that will provide direct access to OpenAI’s **built-in tools**.  


In [ ]:

from openai import OpenAI
client_resp = OpenAI()

# Web search (built-in tool)
resp_ws = client_resp.responses.create(
    model="gpt-4.1",
    messages=[{"role":"user","content":"What was Walmart’s latest quarterly revenue?"}],
    tools=[{"type":"web_search"}]
)
print("WEB SEARCH RESULT:\n", resp_ws.output_text)

# Code interpreter (built-in tool)
resp_ci = client_resp.responses.create(
    model="gpt-4.1",
    messages=[{"role":"user","content":"Calculate the 3-month moving average of [120,150,90,200,130]."}],
    tools=[{"type":"code_interpreter"}]
)
print("CODE INTERPRETER RESULT:\n", resp_ci.output_text)
